In [1]:
// nuget references
#r "nuget: FSharp.Stats, 0.5.1-preview.1"
#r "nuget: Plotly.NET.Interactive, 4.2.1"
#r "nuget: FSharp.Data, 6.3.0"
#r "nuget: Cytoscape.NET.Interactive, 0.2.0"

open FSharp.Stats
open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects
open FSharp.Data
open Cytoscape.NET
open System


//FSharp.Stats.ServiceLocator.setEnvironmentPathVariable (@"C:\Users\bvenn\source\repos\FSharp.Stats\lib")
//FSharp.Stats.Algebra.LinearAlgebra.Service()

//axis styling extension module
module Chart = 
    let myAxis name = LinearAxis.init(Title=Title.init name,Mirror=StyleParam.Mirror.All,Ticks=StyleParam.TickOptions.Inside,ShowGrid=false,ShowLine=true)
    let withAxisTitles x y chart = 
        chart 
        |> Chart.withTemplate ChartTemplates.lightMirrored
        |> Chart.withXAxis (myAxis x) 
        |> Chart.withYAxis (myAxis y)

type Category =
    | Beer
    | Beverage
    | Coffee
    | Other
    with 
        static member FromString (s: string) =
            match s with
            | "Beer" -> Beer
            | "Beverage" -> Beverage
            | "Coffee" -> Coffee
            | _ -> Other

type Order = {
    DateTime    : System.DateTime
    Name        : string
    Gender      : char
    Product     : string
    Price       : float
    Department  : string
    Category    : Category
    Amount      : int
    } with
        static member Create time (name: string) gender product price department category amount = {
            DateTime  = time
            Name      = name
            Gender    = gender
            Product   = product
            Price     = price
            Department= department
            Category  = category
            Amount    = amount
            }

let data = 
    let read =
        CsvFile
            //.Load(@"..\data\coffeedata.txt")
            .Load(@"C:\Users\bvenn\source\repos\brewing-discoveries-workshop\data\coffeedata.txt")
            .Cache()
    read.Rows
    |> Seq.map (fun row -> 
        Order.Create
            (System.DateTime.ParseExact((row.GetColumn "DateTime"),"dd/MM/yyyy HH:mm:ss",null))
            (row.GetColumn "Name")
            (row.GetColumn "Gender" |> char)
            (row.GetColumn "Product")
            (row.GetColumn "Price" |> float) 
            (row.GetColumn "Department")
            ((row.GetColumn "Category") |> Category.FromString)
            (row.GetColumn "Amount" |> int)
        )
    |> Array.ofSeq



Installed Packages Cytoscape.NET.Interactive, 0.2.0 FSharp.Data, 6.3.0 FSharp.Stats, 0.5.1-preview.1 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\bvenn\.nuget\packages\plotly.net.interactive\4.2.1\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

Loading extensions from `C:\Users\bvenn\.nuget\packages\cytoscape.net.interactive\0.2.0\interactive-extensions\dotnet\Cytoscape.NET.Interactive.dll`

## Statistical Testing

Test wether a person drinks significantly more coffee than another one per week.


In [18]:
let coffeTicksVenny =
    data
    |> Array.filter (fun x -> x.Category = Coffee && x.Name = "Benedikt V.")
    |> Array.groupBy (fun x -> x.DateTime.Month,x.DateTime.Year)
    |> Array.map (fun (week, orders) ->
        float orders.Length)
    
coffeTicksVenny
|> Chart.Histogram


<!-- Plotly chart will be drawn inside this DIV -->

In [19]:
let coffeTicksLuk =
    data
    |> Array.filter (fun x -> x.Category = Coffee && x.Name = "Michelle Porter") //Nicholas Thomas
    |> Array.groupBy (fun x -> x.DateTime.Month,x.DateTime.Year)
    |> Array.map (fun (week, orders) ->
        float orders.Length)

coffeTicksLuk
|> Chart.Histogram

<!-- Plotly chart will be drawn inside this DIV -->

In [24]:
let p = Testing.TTest.twoSample false (vector coffeTicksVenny) (vector coffeTicksLuk)

[
    Chart.BoxPlot(Y=coffeTicksVenny,Name="Benedikt V.")
    Chart.BoxPlot(Y=coffeTicksLuk,Name="Michelle Porter")
]
|> Chart.combine
|> Chart.withTitle (string p.PValue)


<!-- Plotly chart will be drawn inside this DIV -->